In [1]:
import numpy as np
import itk
import os
import nibabel as nib
from os import listdir, mkdir
from os.path import isdir, join

In [2]:
# Function to calculate TRE
def TRE(landmarks1, landmarks2, spacing):
    """
    mean (and standard deviation) 3D Euclidean magnitude distance 
    between calculated and reference landmark positions for the 
    set of validation landmarks. All values are reported in units of millimeters.
    
    Parameters:    
        landmarks1 (ndarry): first set of landmarks.
        landmarks2 (ndarry): second set of landmarks.
        spaing (tuple(float)): pixel spacing for x, y, z.
    
    Returns:
        (floar, float) mean and SD 3D Euclidean magnitude distance.
    """
    landmarks1 = spacing*landmarks1
    landmarks2 = spacing*landmarks2
    diff = landmarks1 - landmarks2
    squared = diff * diff
    dist = np.sqrt(np.sum(squared,axis=1))
    mean_TRE = np.mean(dist)
    sd_TRE = np.std(dist)

    return dist, mean_TRE, sd_TRE

#Import transformix text file and convert to np array
def transformix2np(path_transformix, no_points=300):
    """
    Reads and transforms the transformix output to ndarray to use in TRE function
    
    Parameters:    
        path_transformix (string): path to transformix output points txt file
        no_points(int): number of points in transformix file
    
    Returns:
        landmarks_array (ndarray): transformed points 
    """
    import re
    landmarks = open(path_transformix, "r")
    reg_expr = r'OutputIndexFixed = \[([\d.\s\-]+)\]'
    landmarks_array = np.zeros((no_points, 3))

    for index, line in enumerate(landmarks):
        match_obj = re.search(reg_expr, line, re.M)
        coords = match_obj.group(1).split()
        coords = [round(float(c)) for c in coords]
        landmarks_array[index,:] = coords
    return landmarks_array

# Read nii from path and normalize array from 0 to 1
def read_im(image_path):
    nii_img = nib.load(image_path)
    nii_data = nii_img.get_data()
    return nii_data, nii_img

def compute_all_TRE(chosen_param_directory):
    """
    Computes all TRE mean and SD for all four cases and saves to the directory
    "../registration-results/TRE-results"
    
    Parameters:    
        chosen_param_directory (string): directory located inside "../registration-results/" in which 
            results to be evaluated are located
    
    Returns:
        Void
    """
    reg_dir = f"../registration-results/{chosen_param_directory}"
    data_dir = "../data"
    onlydirs = [f for f in listdir(reg_dir) if isdir(join(reg_dir, f))]
    # Array to save TRE results
    TRE_results = np.zeros((len(onlydirs),3))
    TRE_file_path = os.path.join("../registration-results/TRE-results",f"{chosen_param_directory}_TRE_results.csv")

    for idx, chosen_im in enumerate(onlydirs):
        # Load inhale image and get pixel spacing
        path_imge = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.nii.gz')
        inhale_img = nib.load(path_imge)
        # Load inhale image landmarks
        path_landmarksi = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_iBH_xyz_r1.txt")
        landmarksi = np.loadtxt(path_landmarksi)
        # Load transformed parameters
        path_transform_param = f"../registration-results/{chosen_param_directory}/{chosen_im}/outputpoints.txt"
        transformix_landmarks = transformix2np(path_transform_param, no_points=300)
        # Compute TRE
        _, mean_TRE, SD_TRE = TRE(landmarksi, transformix_landmarks, inhale_img.header.get_zooms())
        TRE_results[idx, :] = chosen_im[4], mean_TRE, SD_TRE

    # Write results into folder "../registration-results/TRE-results"
    with open(TRE_file_path, 'w+') as out_f:
        for index, row in enumerate(TRE_results): 
            out_f.write(','.join(str(j) for j in row) + '\n')

In [4]:
# Set chosen image
chosen_im = 'copd1'
chosen_param = 'par0000'

# Set path to data folder containing copd1, copd2, etc. folders
data_dir = "../data"

#Set paths to landmarks and images
path_landmarkse = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_eBH_xyz_r1.txt")
path_landmarksi = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_iBH_xyz_r1.txt")
path_imge = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_eBHCT.nii.gz')
path_imgi = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.nii.gz')

# Set paths to output of transformix
path_transformix = f"../registration-results/{chosen_param}/{chosen_im}/outputpoints.txt"

In [ ]:
# Load landmark points and image
landmarkse = np.loadtxt(path_landmarkse)
landmarksi = np.loadtxt(path_landmarksi)

nii_data, nii_img = read_im(path_imge)

In [ ]:
dist, mean_TRE, SD_TRE = TRE(landmarkse, landmarksi, nii_img.header.get_zooms())
print(mean_TRE, SD_TRE)

In [ ]:
transformix_landmarks = transformix2np(path_transformix, no_points=300)
dist, mean_TRE, SD_TRE = TRE(landmarksi, transformix_landmarks, nii_img.header.get_zooms())
print(mean_TRE, SD_TRE)

## Section to Compute TRE for parameter file

In [16]:
# Compute TRE for four copd images for a chosen_parameter file
# Result is saved in folder "./registration-results/TRE-results"
chosen_param = 'Par0035-MI-ASGDPrime-both-hist256-grid6'

In [17]:
reg_dir = f"../registration-results/{chosen_param}"
data_dir = "../data"
onlydirs = [f for f in listdir(reg_dir) if isdir(join(reg_dir, f))]
# Array to save TRE results
TRE_results = np.zeros((len(onlydirs),3))
TRE_file_path = os.path.join("../registration-results/TRE-results",f"{chosen_param}_TRE_results.csv")

for idx, chosen_im in enumerate(onlydirs):
    # Load inhale image and get pixel spacing
    path_imge = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.nii.gz')
    inhale_img = nib.load(path_imge)
    # Load inhale image landmarks
    path_landmarksi = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_iBH_xyz_r1.txt")
    landmarksi = np.loadtxt(path_landmarksi)
    # Load transformed parameters
    path_transform_param = f"../registration-results/{chosen_param}/{chosen_im}/outputpoints.txt"
    transformix_landmarks = transformix2np(path_transform_param, no_points=300)
    # Compute TRE
    _, mean_TRE, SD_TRE = TRE(landmarksi, transformix_landmarks, inhale_img.header.get_zooms())
    TRE_results[idx, :] = chosen_im[4], mean_TRE, SD_TRE

# Write results into folder "../registration-results/TRE-results"
with open(TRE_file_path, 'w+') as out_f:
    for index, row in enumerate(TRE_results): 
        out_f.write(','.join(str(j) for j in row) + '\n')

## Section to Write Elastix Commands to file

In [18]:
# Write commands for elastix test set registration
# Add parameter folder name and paramter file names (must be saved in ./parameter-files folder)
param_file = 'Par0035-MI-ASGDPrime-both-hist256-ss10000' # folder name where results will be saved
param_1 = 'Par0035.SPREAD.MI.af.0.txt'
param_2 = 'Par0035.SPREAD.MI.bs.1.ASGDPrime.txt'
data_dir = "../data"
test_ims = [f for f in listdir(data_dir) if isdir(join(data_dir, f))]
result_folder = f"../registration-results/{param_file}/"

In [6]:
def get_elastix_transformix_file(result_dir_name, param_af, param_bs, data_dir):
    """
    Write commands for elastix and transformix test set registration.
    
    Args:
        result_dir_name (str): subfolder name for storing registration results inside ./registration-results.
        param_af (str): affine registration parameters file name, must be saved in ./parameter-files folder.
        param_bs (str): B-splines registration parameters file name, must be saved in ./parameter-files folder.
        data_dir (str): the data that will be registered.
    """
    test_ims = [f for f in listdir(data_dir) if isdir(join(data_dir, f))]
    result_folder = f"../registration-results/{result_dir_name}/"

    mkdir(result_folder)

    with open(f"elastix_transformix_{result_dir_name}", 'w+') as out_f:

        for chosen_im in test_ims:
            fixed_im_path = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.nii.gz') 
            moving_im_path = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_eBHCT.nii.gz')
            result_path = f"../registration-results/{result_dir_name}/{chosen_im}/"
            param_af_path = f"../parameter_files/{param_af}"
            param_bs_path = f"../parameter_files/{param_bs}"    
            mkdir(result_path)
            cmd = f"elastix -f {fixed_im_path} -m {moving_im_path} -out {result_path} -p {param_af_path} -p {param_bs_path}\n"
            out_f.write(cmd)

            def_path = f"../data/{chosen_im}/{chosen_im}_300_iBH_xyz_r1_elastix.txt"
            result_path = f"../registration-results/{result_dir_name}/{chosen_im}/"
            parameters_path = f"../registration-results/{result_dir_name}/{chosen_im}/TransformParameters.1.txt"
            cmd = f"transformix -def {def_path} -out {result_path} -tp {parameters_path}\n"
            out_f.write(cmd)


def replace(file_path, pattern, subst):
    """
    Replace strings in a file.
    
    Parametrs:
        pattern (str, iterable): pattern to replace.
        subst (str, iterable): subtitution.
    
    """
    from os import fdopen, remove
    from shutil import move
    from tempfile import mkstemp

    #Create temp file
    fh, abs_path = mkstemp()
    with fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

In [19]:
# Write commands for transformix transformation
with open(f"test_transformation_{param_file}", 'w+') as out_f:
    
    for chosen_im in test_ims:
        def_path = f"../data/{chosen_im}/{chosen_im}_300_iBH_xyz_r1_elastix.txt"
        result_path = f"../registration-results/{param_file}/{chosen_im}/"
        parameters_path = f"../registration-results/{param_file}/{chosen_im}/TransformParameters.0.txt"
        cmd = f"transformix -def {def_path} -out {result_path} -tp {parameters_path}\n"
        out_f.write(cmd)

# Change parameters line by line

In [14]:
# Write commands for elastix test set registration
# Add parameter folder name and paramter file names (must be saved in ./parameter-files folder)
result_dir_name = 'Par0035-MI-ASGDPrime-both-hist256-grid6' # folder name where results will be saved
param_af = 'Par0035.SPREAD.MI.af.0.txt'
param_bs = 'Par0035.SPREAD.MI.bs.1.ASGDPrime.txt'
data_dir = "../data"

#### Which parmeter to change:
* (NumberOfResolutions 3) -> 5 & (ImagePyramidSchedule  4 4 4 2 2 2 1 1 1) -> 16 16 16 8 8 8 4 4 4 2 2 2 1 1 1 & (GridSpacingSchedule 4.0 2.0 1.0) -> 16.0 8.0 4.0 2.0 1.0
* (NumberOfHistogramBins 32) -> 64, 128, 256
* (MaximumNumberOfIterations 500) -> 1000
* (NumberOfSpatialSamples 5000) -> 7500
* (FinalBSplineInterpolationOrder 3) -> ???
* (FinalGridSpacingInPhysicalUnits 10.0 10.0 10.0) -> (FinalGridSpacingInPhysicalUnits 5.0 5.0 5.0)
* (FinalGridSpacingInPhysicalUnits 10.0 10.0 10.0) -> (FinalGridSpacingInPhysicalUnits 20.0 20.0 20.0)

In [15]:
from shutil import copyfile

# Change affine parameters
test_param_af = param_af
af_param_file_path = f"../parameter_files/{test_param_af[:-4]}-working.txt"
af_working_param_file = f"{test_param_af[:-4]}-working.txt"
copyfile(f"../parameter_files/{test_param_af}", af_param_file_path)
replace(af_param_file_path, "(NumberOfHistogramBins 32)", "(NumberOfHistogramBins 256)")

# Change bspline parameters
test_param_bs = param_bs
bs_param_file_path = f"../parameter_files/{test_param_bs[:-4]}-working.txt"
bs_working_param_file = f"{test_param_bs[:-4]}-working.txt"
copyfile(f"../parameter_files/{test_param_bs}", bs_param_file_path)
replace(bs_param_file_path, "(NumberOfHistogramBins 32)", "(NumberOfHistogramBins 256)")
replace(bs_param_file_path, "(FinalGridSpacingInPhysicalUnits 10.0 10.0 10.0)", "(FinalGridSpacingInPhysicalUnits 6.0 6.0 6.0)")


get_elastix_transformix_file(result_dir_name, af_working_param_file, bs_working_param_file, data_dir)